In [2]:
import pickle
import os
my_list=[1,2]
e=1
# 假设列表为 my_list
file_path = os.path.join('../result/', f'my_list_{e}.pkl')  # 文件路径为 result/my_list.pkl

with open(file_path, 'wb') as f:
    pickle.dump(my_list, f)



In [2]:
import torch
import numpy as np
import math

def generate_grads_with_privacy_ldp(grads, num_selected, clip_norm, epsilon):
    if num_selected > 0:
        total_norm = torch.norm(torch.stack([torch.norm(g.detach()) for g in grads]))
        clip_coef = clip_norm / (total_norm + 1e-6)
        if clip_coef < 1:
            grads = [g * clip_coef for g in grads]

    for i in range(len(grads)):
        grads[i] = grads[i] + np.random.laplace(0, clip_norm/epsilon, size=grads[i].shape)

    return grads

# 梯度列表
grads = [torch.rand((5, 5)), torch.rand((5, 5))]

# 选定的裁剪数
num_selected = 2

# 裁剪范数
clip_norm = 0.1

# 隐私预算
epsilon = 0.1

# 添加隐私
private_grads = generate_grads_with_privacy_ldp(grads, num_selected, clip_norm, epsilon)
private_grads


[tensor([[-0.0253,  2.4540, -0.9299,  0.3203,  1.2779],
         [ 0.0957, -0.0838,  6.0922,  1.5990,  0.4150],
         [-0.7917, -1.7844, -2.7360,  0.0079, -3.1418],
         [ 1.0139, -2.5167,  0.0997,  2.0076,  0.3560],
         [-0.3350, -0.1313,  1.4936,  0.1405, -0.5117]], dtype=torch.float64),
 tensor([[ 0.4154, -0.1231, -0.0840,  3.3802,  0.0212],
         [ 2.1378, -0.2380,  2.1818,  2.8994, -0.0779],
         [-0.0632,  0.2164,  0.4578, -0.0243, -1.8013],
         [-0.3919,  0.0058,  0.8350,  3.4534, -3.8682],
         [-0.1007,  0.7516, -0.9256, -0.6446,  1.4835]], dtype=torch.float64)]

In [2]:
# 0.5的隐私预算下，4次cdp/ldp/真实 的梯度统计。 shapley值统计
# 然后进行均值 然后进行mse差值比较
for epsilon in range(5, 21, 5):
    epsilon = round(epsilon / 10, 1)
    print(epsilon)

0.5
1.0
1.5
2.0
